#### 파일 실행 위치: input/code/make_newdataset.ipynb

In [348]:
import os
import json
import warnings
import copy
warnings.filterwarnings('ignore')

In [349]:
test_new = []
test_original = []
for i in range(len(dataset_test_new['images'])):
    test_new.append(dataset_test_new['images'][i]['file_name']) # 624
for i in range(len(dataset_test_original['images'])):
    test_original.append(dataset_test_original['images'][i]['file_name']) # 819
    
test_leak = list(set(test_original) - set(test_new)) # 유출된 159장 filename
# print(len(test_new))
# print(len(test_original))
# print(len(test_leak))

In [350]:
dataset_path  = '../data'
train_path = dataset_path + '/train.json'
train_all_path = dataset_path + '/train_all.json'
data_leak_path = dataset_path + '/batch_03/data.json'

with open(train_path, 'r') as f:
    dataset_train = json.loads(f.read())
with open(train_all_path, 'r') as f:
    dataset_train_all = json.loads(f.read())
with open(data_leak_path, 'r') as f:
    dataset_leak = json.loads(f.read())
dataset_leak_copy = copy.deepcopy(dataset_leak)

In [351]:
# print(len(dataset_train['images']))
# print(len(dataset_train['annotations'])) # last id: 21115
# print(len(dataset_train_all['images']))
# print(len(dataset_train_all['annotations'])) # last id: 26401

In [352]:
# merge train.json
new_images = []
new_annotations = []
img_id = 2617
anno_id = 21116
for i in range(len(dataset_leak['images'])):
    if dataset_leak['images'][i]['file_name'] in test_leak:
        for j in range(len(dataset_leak['annotations'])):
            if dataset_leak['annotations'][j]['image_id'] == i and dataset_leak['annotations'][j]['category_id'] != 0:
                dataset_leak['annotations'][j]['image_id'] = img_id
                dataset_leak['annotations'][j]['id'] = anno_id
                new_annotations.append(dataset_leak['annotations'][j])
                anno_id += 1
        dataset_leak['images'][i]['id'] = img_id
        new_images.append(dataset_leak['images'][i])  
        img_id += 1
        
del new_annotations[267] # segment labeling 안되어있는것 제거

In [353]:
# merge train_all.json
new_images_all = []
new_annotations_all = []
img_id = 3272
anno_id = 26402
for i in range(len(dataset_leak_copy['images'])):
    if dataset_leak_copy['images'][i]['file_name'] in test_leak:
        for j in range(len(dataset_leak_copy['annotations'])):
            if dataset_leak_copy['annotations'][j]['image_id'] == i and dataset_leak_copy['annotations'][j]['category_id'] != 0:
                dataset_leak_copy['annotations'][j]['image_id'] = img_id
                dataset_leak_copy['annotations'][j]['id'] = anno_id
                new_annotations_all.append(dataset_leak_copy['annotations'][j])
                anno_id += 1
        dataset_leak_copy['images'][i]['id'] = img_id
        new_images_all.append(dataset_leak_copy['images'][i])  
        img_id += 1
del new_annotations_all[267] # segment labeling 안되어있는것 제거

In [354]:
# train
dataset_train_new_images = dataset_train['images'] + new_images
# print(len(dataset_train['images']),len(new_images),len(dataset_train['images'])+len(new_images))
# print(len(dataset_train_new_images))

dataset_train_new_annotations = dataset_train['annotations'] + new_annotations
# print(len(dataset_train['annotations']),len(new_annotations),len(dataset_train['annotations'])+len(new_annotations))
# print(len(dataset_train_new_annotations))

# train_all
dataset_train_all_new_images = dataset_train_all['images'] + new_images_all
# print(len(dataset_train_all['images']),len(new_images_all),len(dataset_train_all['images'])+len(new_images_all))
# print(len(dataset_train_all_new_images))

dataset_train_all_new_annotations = dataset_train_all['annotations'] + new_annotations_all
# print(len(dataset_train_all['annotations']),len(new_annotations_all),len(dataset_train_all['annotations'])+len(new_annotations_all))
# print(len(dataset_train_all_new_annotations))

2617 195 2812
2812
20988 691 21679
21679
3272 195 3467
3467
26240 691 26931
26931


In [355]:
dataset_train['images'] = dataset_train_new_images
dataset_train['annotations'] = dataset_train_new_annotations
# print(len(dataset_train['images']))
# print(len(dataset_train['annotations']))

dataset_train_all['images'] = dataset_train_all_new_images
dataset_train_all['annotations'] = dataset_train_all_new_annotations
# print(len(dataset_train_all['images']))
# print(len(dataset_train_all['annotations']))

2812
21679
3467
26931


In [356]:
save_path1 = '../data/train_new.json'
with open(save_path1, 'w', encoding='utf-8') as f:
    json.dump(dataset_train, f)

save_path2 = '../data/train_all_new.json'
with open(save_path2, 'w', encoding='utf-8') as f:
    json.dump(dataset_train_all, f)